In [14]:
from lib.imports import *
import os

In [15]:
normalize = True
balance = False
centered = True

window_size = 501
stride = 1 # in datapoints
block_size = 1

train_size = .5
dev_size = .25
test_size = .25
n_files = 2
n_holdout_files = 1

files = list(range(71))
skip_idx = [19,24,26,32,34,38,40,45,52,55,70]
for idx in skip_idx:
    files.remove(idx)
fileindices = random.sample(files,n_files)
holdout_indices = fileindices[(n_files-n_holdout_files):]
print(fileindices,holdout_indices)

[43, 35] [35]


In [16]:
from lib.utils import load_nursing_by_index

In [17]:
X,y = load_nursing_by_index(0)
torch.bincount(y.long())
# import plotly.express as px
# df = pd.DataFrame([X[:,0].numpy(),y.argmax(axis=1).numpy()]).T
# fig = px.line(df)
# fig.show(renderer='browser')

tensor([138679,   4321])

In [18]:
y_train_all = torch.Tensor()
y_dev_all = torch.Tensor()
y_test_all = torch.Tensor()
x_train_i = 0
x_dev_i = 0
x_test_i = 0
data_dir = f'w{window_size}_centered_tiny'
os.makedirs(data_dir)
os.makedirs(f'{data_dir}/train')
os.makedirs(f'{data_dir}/dev')
os.makedirs(f'{data_dir}/test')
config = {
    'TRAIN_SIZE':train_size,
    'DEV_SIZE':dev_size,
    'TEST_SIZE':test_size,
    'FILES':fileindices,
    'NORMALIZED':normalize,
    'BALANCED':balance,
    'HOLDOUT_FILES':holdout_indices
}
with open(f'{data_dir}/config.json', 'w') as f:
     f.write(json.dumps(config))

In [19]:
for fileindex in fileindices:
    X,y_true = load_nursing_by_index(fileindex)

    if (fileindex in holdout_indices):
        torch.save((X,y_true),f'{data_dir}/holdout_{fileindex}.pt')
        continue
    
    x = X[:,0].unsqueeze(1)
    y = X[:,1].unsqueeze(1)
    z = X[:,2].unsqueeze(1)
    xs = [x[:-(window_size-1)]]
    ys = [y[:-(window_size-1)]]
    zs = [z[:-(window_size-1)]]
    for i in range(1,window_size-1):
        xs.append(x[i:i-(window_size-1)])
        ys.append(y[i:i-(window_size-1)])
        zs.append(z[i:i-(window_size-1)])
    xs.append(x[(window_size-1):])
    ys.append(y[(window_size-1):])
    zs.append(z[(window_size-1):])
    xs = torch.cat(xs,axis=1).float()
    ys = torch.cat(ys,axis=1).float()
    zs = torch.cat(zs,axis=1).float()
    X = torch.cat([xs,ys,zs],axis=1)
    y_true = y_true[window_size//2:-(window_size//2)].unsqueeze(1)

    # train test split
    X_train,X_test,y_train,y_test = train_test_split(X,y_true,test_size=(1-train_size),shuffle=True,stratify=y_true,random_state=0)
    X_dev,X_test,y_dev,y_test = train_test_split(X_test,y_test,test_size=(test_size/(1-train_size)),shuffle=True,stratify=y_test,random_state=0)

    y_train_all = torch.cat([y_train_all,y_train])
    y_dev_all = torch.cat([y_dev_all,y_dev])
    y_test_all = torch.cat([y_test_all,y_test])

    for Xi in X_train:
        torch.save(Xi.clone(),f'{data_dir}/train/{x_train_i}.pt')
        x_train_i += 1
    for Xi in X_dev:
        torch.save(Xi.clone(),f'{data_dir}/dev/{x_dev_i}.pt')
        x_dev_i += 1
    for Xi in X_test:
        torch.save(Xi.clone(),f'{data_dir}/test/{x_test_i}.pt')
        x_test_i += 1
torch.save(y_train_all,f'{data_dir}/y_train.pt')
torch.save(y_dev_all,f'{data_dir}/y_dev.pt')
torch.save(y_test_all,f'{data_dir}/y_test.pt')

In [20]:
# test if worked
from lib.datasets import Dataset2p0
from torch.utils.data import DataLoader
trainloader = DataLoader(Dataset2p0(dir=f'{data_dir}/train/',labels=f'{data_dir}/y_train.pt'),batch_size=1,shuffle=True)
devloader = DataLoader(Dataset2p0(dir=f'{data_dir}/dev/',labels=f'{data_dir}/y_test.pt'),batch_size=1,shuffle=True)
testloader = DataLoader(Dataset2p0(dir=f'{data_dir}/test/',labels=f'{data_dir}/y_test.pt'),batch_size=1,shuffle=True)
X,y = next(iter(trainloader))
X,y = next(iter(devloader))
X,y = next(iter(testloader))
print(X.shape,y.shape)
print(len(trainloader)+len(devloader)+len(testloader))

torch.Size([1, 1503]) torch.Size([1, 1])
144000


In [21]:
y

tensor([[0.]], dtype=torch.float64)